In [8]:
from dotenv import load_dotenv
import os
load_dotenv()

from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage
from operator import itemgetter
from langchain_core.messages.utils import trim_messages

In [2]:

groq_api_key = os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)

In [3]:
# Store chat history
store = {}
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [4]:
# Define prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful and concise assistant. Answer based on the full conversation history."),
    MessagesPlaceholder(variable_name="messages"),
    ("human", "{input}")
])


In [9]:


trimmer = trim_messages(
    max_tokens=600,  # Adjusted slightly for demo, original was 45 which is very small
    strategy="last",
    token_counter=model, # Uses the model's tokenizer for accurate counting
    include_system=True,
    allow_partial=False,
    start_on="human" # Start counting from the first human message
)

In [11]:
from langchain_core.runnables import RunnablePassthrough

# Core chain (combines prompt and model)
chain = (
        RunnablePassthrough.assign( # Allows you to modify specific parts of the input
            messages=itemgetter("messages") | trimmer # Apply trimmer only to the 'messages' part
        )
        | prompt # Format the input according to the prompt template
        | model # Send the formatted prompt to the AI model
)


In [12]:
# Chain with memory
final_chain = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="messages",
)

In [13]:
# --- Example Chat Session ---
session_config = {"configurable": {"session_id": "chat_alpha"}}

# First question
response1 = final_chain.invoke(
    {"input": "What is self-reflection?"},
    config=session_config
)
print(f"User: What is self-reflection?\nAI: {response1.content}\n")

# Follow-up
response2 = final_chain.invoke(
    {"input": "What are a few ways to achieve that?"},
    config=session_config
)
print(f"User: What are a few ways to achieve that?\nAI: {response2.content}\n")

# Memory test
response3 = final_chain.invoke(
    {"input": "Can you remind me what I asked first?"},
    config=session_config
)
print(f"User: Can you remind me what I asked first?\nAI: {response3.content}\n")


User: What is self-reflection?
AI: Self-reflection is the process of thoughtfully considering your own thoughts, feelings, actions, and motivations. It involves examining your inner world and gaining a deeper understanding of yourself.  


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


User: What are a few ways to achieve that?
AI: Here are a few ways to practice self-reflection:

* **Journaling:** Regularly writing down your thoughts and feelings can help you identify patterns and gain clarity.
* **Meditation:**  Focusing on your breath and observing your thoughts without judgment can increase self-awareness.
* **Mindfulness:** Paying attention to the present moment and your experiences can help you understand your reactions and motivations.
* **Asking for feedback:** Seeking honest opinions from trusted friends or mentors can offer valuable insights into your strengths and weaknesses.
* **Reflecting on experiences:**  Taking time to analyze past events, both positive and negative, can help you learn and grow. 



Let me know if you'd like to explore any of these methods in more detail! 

User: Can you remind me what I asked first?
AI: You first asked: "What is self-reflection?".  😊 


In [14]:
response4 = final_chain.invoke(
    {"input" : "You did mention a few ways to achieve self reflection can you expand more on those topics"},
    config=session_config
)
print(f"User:You did mention a few ways to achieve self reflection can you expand more on those topics \n AI: {response4} \n")

User:You did mention a few ways to achieve self reflection can you expand more on those topics 
 AI: content="You got it! Let's dive deeper into those self-reflection techniques:\n\n**1. Journaling:**\n\n* **How:** Set aside a specific time each day (even 5-10 minutes) to write freely.  Don't worry about grammar or structure, just let your thoughts flow. \n* **What to write about:**\n    * Your daily experiences, both big and small.\n    * Your feelings and emotions, exploring what triggered them.\n    * Your thoughts about specific situations or challenges.\n    * Your goals and aspirations, reflecting on your progress.\n    * Anything else that comes to mind.\n\n* **Benefits:** Journaling helps you process emotions, identify patterns in your thinking, and gain clarity on your values and beliefs.\n\n**2. Meditation:**\n\n* **How:** Find a quiet space, sit comfortably, and focus on your breath. When your mind wanders (and it will!), gently bring your attention back to your breath. Ther